covered topics 
- details about generator 
- generator *comprehension* / *func* / *class*
- bundled generators 
- coroutines

things I didn't do
- pros and cons about *generator*
- fully understand the examples of *coroutines* 

### generators

count func

In [52]:
def count(start=0, step=1, end=10):
    n = start 
    while n <= end:
        yield n 
        n += step 

        
for x in count(10, 2.5, 15):
    x 

10

12.5

15.0

count class

In [53]:
class Count(object):
    def __init__(self, start=0, step=1, end=10):
        self.n    = start 
        self.step = step 
        self.end  = end  
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        n = self.n 
        if n > self.end:
            raise StopIteration()
            
        self.n += self.step 
        return n 
    
    
for x in Count(10, 2.5, 15):
    x 

10

12.5

15.0

yield, return 

In [54]:
def generator():
    yield "hey, Mr. Y"
    return 'returning from ... what'


g = generator()

try:
    next(g)  # result returned by `yield`
    next(g)  # the final was returned by `return` 
except StopIteration as err:
    err 

'hey, Mr. Y'

StopIteration('returning from ... what')

comprehension

In [55]:
list(
    x**3 for x in range(5)
)

[0, 1, 8, 27, 64]

order

In [56]:
def gene():
    
    print('Before 1')
    yield 1 
    
    print('After  1')
    print('Before 2')
    yield 2 
    
    print('After  2')
    print('Before 3')
    yield 3 
    
    print('After  3')
    
    '''
        if `return` or None:
            nothing returned 
        
        if `yield` 
            yield what u've given
    '''
    return 'hela '


g = gene()

for i in g:  # u can use `next` instead 
    i 

Before 1


1

After  1
Before 2


2

After  2
Before 3


3

After  3


pipelines

In [57]:
!cat ./src/p5_gen_pipe.txt | grep spam 

!cat ./src/p5_gen_pipe.txt | grep spam | sed 's/spam/bacon/g'

spam
spam spam 
spam spam spam 
bacon
bacon bacon 
bacon bacon bacon 


In [58]:
# recreating the pipe with generators :)

def cat(filename):
    for line in open(filename):
        yield line.rstrip()
        
def grep(sequence, search):
    for line in sequence:
        if search in line:
            yield line 
            
def replace(sequence, _search, _replace):
    for line in sequence:
        yield line.replace(_search, _replace)

In [59]:
seperated_steps = '''
    lines       = cat('./src/p5_gen_pipe.txt')
    spam_lines  = grep(lines, 'spam')
    bacon_lines = replace(spam_lines, 'spam', 'bacon')
'''

# compressed in one line XD

for line in replace(
    grep(cat('./src/p5_gen_pipe.txt'), 'spam'), 
    'spam',
    'bacon'
):
    print(line)

bacon
bacon bacon
bacon bacon bacon


tee - what does it do

In [60]:
import itertools as itr 

def spam_and_eggs():
    yield 'spam'
    yield 'eggs'
    

a, b, c = itr.tee(spam_and_eggs(), 3)

# the func is not that important 
#   the crucial one is the `n` (latter args)
#   it's like repeated genrators? (I think)
a, b, c 

[ _ for _ in a ]
[ _ for _ in b ]
[ _ for _ in c ]

(<itertools._tee at 0x10f0477c8>,
 <itertools._tee at 0x10f0c9d08>)

['spam', 'eggs']

['spam', 'eggs']

['spam', 'eggs']

tee - actual (sort of) implementataion

In [61]:
from collections import deque

def _tee(iterable, n=2):
    
    itr = iter(iterable)
    deq = [deque() for i in range(n)] 
    
    def gen(mydeq):
        
        while True:
            
            if not mydeq:
                
                try:
                    newval = next(itr)
                except StopIteration:
                    return 
                
                for d in deq:
                    d.append(newval)
            
            yield mydeq.popleft()
            
    return tuple(gen(d) for d in deq)


a, b, c = _tee(spam_and_eggs(), 3)

[i for i in a]
[i for i in b]
[i for i in c]

['spam', 'eggs']

['spam', 'eggs']

['spam', 'eggs']

powerset

In [62]:
import itertools as itr 

def powerset(seq):
    for size in range(1, len(seq) + 1):
        for item in itr.combinations(seq, size): 
            yield item 
            

for res in powerset('abc'):
    res 

('a',)

('b',)

('c',)

('a', 'b')

('a', 'c')

('b', 'c')

('a', 'b', 'c')

powerset - a simpler one

In [63]:
def powerset(seq):
    for size in range(1, len(seq) + 1):
        yield from itr.combinations(seq, size)  # for-yield => 1 line


for res in powerset('abc'):
    res

('a',)

('b',)

('c',)

('a', 'b')

('a', 'c')

('b', 'c')

('a', 'b', 'c')

flatten a list *recursively*

In [64]:
def flatten(seq):
    for item in seq:
        try:
            yield from flatten(item)
        except TypeError:
            yield item
            
            
list(flatten(
    [[], 1, [3, 5, 7], [9]]
))

[1, 3, 5, 7, 9]

context managers

In [65]:
# stage - init 

from datetime import datetime

from contextlib import (
    contextmanager, redirect_stdout,
    ExitStack
)


@contextmanager
def timer(name):
    
    start = datetime.now()
    yield 
    end   = datetime.now()
    
    print(f'{name} took {end-start}')

In [66]:
# stage - exp - ver1

@contextmanager
def write_to_log(name):
    with open(f'./src/p5_contx_{name}.txt', 'w') as fh:
        with redirect_stdout(fh):
            with timer(name):
                yield 
                
                
# then use it 

@write_to_log('some_func')
def some_func():
    print('hmm... hello?')
    print('and... hi?')
    
some_func()
 

!cat src/p5_contx_some_func.txt

hmm... hello?
and... hi?
some_func took 0:00:00.000016


context managers - ```ExitStack```

In [67]:
# stage - exp - ver2

@contextmanager
def write_to_log(name):
    
    with ExitStack() as stack:
        
        fh = stack.enter_context(
            open('./src/p5_contx_exstack.txt', 'w'))
        
        stack.enter_context(redirect_stdout(fh))
        stack.enter_context(timer(name))
        
        yield 
        

@write_to_log('another func')
def what_the_func():
    print('hey yo')
    print('whats up')

what_the_func()


!cat src/p5_contx_exstack.txt

hey yo
whats up
another func took 0:00:00.000025


In [68]:
# stage - exp - ver3 (maybe not)

# shortten the filename in code 
spam_file = './src/p5_contx_spam.txt'
eggs_file = './src/p5_contx_eggs.txt'


with ExitStack() as stack:
    spam_fh = stack.enter_context(open(spam_file, 'w'))
    eggs_fh = stack.enter_context(open(eggs_file, 'w'))
    
    spam_bytes_written = spam_fh.write('whoo~ spam')
    eggs_bytes_written = eggs_fh.write('whoo~ eggs')
    
    close_handlers = stack.pop_all().close
   

spam_bytes_written = spam_fh.write('yet another spam')
eggs_bytes_written = eggs_fh.write('and another eggs')


# then ...
close_handlers()

try:
    spam_bytes_written = spam_fh.write('cant write now')
except ValueError as err:
    err 

ValueError('I/O operation on closed file.')

### coroutines 

basic one

In [69]:
def generator():
    
    val = yield 'spam' 
    
    print(f'received val: {val}')
    yield f'previous val: {val}'
    

gn = generator()

next(gn)  # yielded val 
next(gn)  # 其与 return 同等级, 也许后边的 val 需再次被赋值

gs = generator()

gs.send(None)     # the first one must be `None`
gs.send('whooo')  # 原理类似 next (另将值"送"进 generator)

'spam'

received val: None


'previous val: None'

'spam'

received val: whooo


'previous val: whooo'

a decorator we're gonna use for many times

In [70]:
from functools import wraps 

def coroutine(func):
    
    @wraps(func)
    def _coroutine(*args, **kwargs):
        active_coroutine = func(*args, **kwargs)
        next(active_coroutine)
        
        return active_coroutine
    
    return _coroutine

example: about ```send```

In [71]:
@coroutine
def spam():
    while True:
        print('Waiting ...\n')
        
        val = yield 
        print(f'Got {val}')


gen = spam()  # Waiting 

gen.send('wheee')
gen.send('whooo')

Waiting ...

Got wheee
Waiting ...

Got whooo
Waiting ...



example: ```close``` & ```throw```

In [72]:
@coroutine
def simple_coroutine():
    
    print('Setting up!\n')
    
    try:
        while True:
            item = yield 
            print(f'Got: {item}\n')
    
    except GeneratorExit:
        print('Normal exit')
        
    except Exception as e:
        print(f'Excptn exit: {e!r}')
        raise
        
    finally:
        print('Any exit.')

In [73]:
asc = simple_coroutine()

asc.send('Spam')
asc.send('Wooo')

asc.close()

Setting up!

Got: Spam

Got: Wooo

Normal exit
Any exit.


In [79]:
ast = simple_coroutine()

try:
    ast.send('Whoo')
    ast.send('Yooo')
    
    ast.throw(RuntimeError, 'Oops!')
except:
    pass 

Setting up!

Got: Whoo

Got: Yooo

Excptn exit: RuntimeError('Oops!',)
Any exit.


example: bidirectional pipelines

In [86]:
# rewrite the pipeline we've written before :)

!cat ./src/p5_gen_pipe.txt

the_file = './src/p5_gen_pipe.txt'

spam
eggs 
spam spam 
eggs eggs 
spam spam spam 
eggs eggs eggs

In [87]:
# rewrite part of it 

@coroutine
def replace(_search, _replace):
    while True:
        item = yield 
        print(item.replace(_search, _replace))
        
        
spam_replace = replace('spam', 'bacon')

for line in open(the_file):
    spam_replace.send(line.rstrip())  # the original stays!

bacon
eggs
bacon bacon
eggs eggs
bacon bacon bacon
eggs eggs eggs


In [92]:
# fully rewrite!

@coroutine
def grep(target, pattern):
    while True:
        item = yield 
        if pattern in item:
            target.send(item)
            
@coroutine
def replace(target, _search, _replace):
    while True:
        target.send((yield).replace(_search, _replace))
        
@coroutine
def _print(fmtstring):
    while True:
        print(fmtstring % (yield))
        
@coroutine
def tee(*targets):
    while True:
        item = yield 
        for target in targets:
            target.send(item)

In [96]:
printer = _print('%s')

replacer_spam = replace(printer, 'spam', 'bacon')
replacer_eggs = replace(printer, 'spam spam', 'sausage')

branch  = tee(replacer_spam, replacer_eggs)

grepper = grep(branch, 'spam') 


for line in open(the_file):
    grepper.send(line.rstrip())

bacon
spam
bacon bacon
sausage
bacon bacon bacon
sausage spam


example: using the *state*

In [115]:
# simple version

@coroutine
def average():
    
    count = 1 
    total = yield 
    
    while True:
        total += yield total / count 
        count += 1 
        

avg = average()

avg.send(20)
avg.send(5)
avg.send(11)

20.0

12.5

12.0

In [127]:
# and another version

@coroutine
def _print(fmtstring):
    while True:
        print(fmtstring % (yield))

@coroutine
def average(target):
    
    count = 0
    total = 0 
    
    while True:
        count += 1 
        total += yield 
        
        target.send(total / count)
        
        
printer = _print('%.1f')
avg     = average(printer)

avg.send(20)
avg.send(5)
avg.send(11) 

20.0
12.5
12.0


example: about ```groupby```

In [137]:
# generator_coroutine version 

@coroutine
def groupby():
    
    key, val      = yield 
    old_key, vals = key, []
    
    while True:
        
        old_val = val 
        
        if key == old_key:
            key, val = yield 
            
        else:
            key, val      = yield old_key, vals 
            old_key, vals = key, []
            
        vals.append(old_val)

In [149]:
gb = groupby()

gb.send(('a', 1))
gb.send(('a', 3))
gb.send(('a', 5))
gb.send(('b', 8))      # detected! XD

gb.send(('b', 10))
gb.send(('a', 11))     # detected! XD

gb.send(('a', 13))
gb.send((None, None))  # detected! XD

('a', [1, 3, 5])

('b', [8, 10])

('a', [11, 13])

In [150]:
# pure_coroutine version

@coroutine
def _print(fmtstring):
    while True:
        print(fmtstring % (yield))

@coroutine
def groupby(target):
    
    old_key = None 
    
    while True:
        
        key, val = yield 
        
        if old_key != key:
            
            if old_key and vals:
                target.send((old_key, vals))
            
            vals    = []
            old_key = key 
        
        vals.append(val)

In [161]:
gp = groupby(_print(
    '[%s] -- %s'  # group -- values 
))

gp.send(('a', 1))
gp.send(('a', 3))
gp.send(('a', 5))
gp.send(('b', 8))      # detected! XD

gp.send(('b', 10))
gp.send(('b', 20))
gp.send((None, None))  # detected! XD

[a] -- [1, 3, 5]
[b] -- [8, 10, 20]


<hr>

***The End***.